In [1]:
import pandas as pd
import numpy as np # linear algebra

# Replace 'path_to_dataset' with the actual path to your dataset file
df = pd.read_csv('Dataset33new (1).csv')
df.shape

(13719, 16)

In [2]:
X = df.drop(columns=['Brand'])
y = df['Brand']

categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['int', 'float']).columns

# Create DataFrames for categorical and numerical columns
X_categorical = X[categorical_columns]
X_numerical = X[numerical_columns]

from sklearn.preprocessing import LabelEncoder



# Perform label encoding only on columns with dtype object
label_encoder = LabelEncoder()
X_encoded = X_categorical.copy()
for col in categorical_columns:
    X_encoded[col] = X[col].astype(str)  # Ensure the column is of string dtype before label encoding
    X_encoded[col] = label_encoder.fit_transform(X_encoded[col])

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


# Initialize LabelEncoder for the target variable y
label_encoder_y = LabelEncoder()
y_encoded = label_encoder_y.fit_transform(y)
# Merge with original X containing int and float columns
X_final = pd.concat([X_encoded, X.select_dtypes(include=['int', 'float'])], axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_encoded, test_size=0.2, random_state=42)

In [3]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE

smote = SMOTE(random_state=42)
X_train_resampled_smote, y_train_resampled_smote = smote.fit_resample(X_train, y_train)

borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled_borderline, y_train_resampled_borderline = borderline_smote.fit_resample(X_train, y_train)


## SMOTE + Bordeline SMOTE

RF

In [4]:
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the classifier
rf_classifier.fit(X_train_resampled_smote, y_train_resampled_smote)
# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_rf_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_rf = best_rf_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
classification_report_result_best_rf = classification_report(y_test, y_pred_best_rf, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Random Forest with SMOTE): {accuracy_best_rf}")
print("Classification Report (Best Random Forest with SMOTE):\n", classification_report_result_best_rf)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'max_depth': None, 'n_estimators': 200}
Accuracy (Best Random Forest with SMOTE): 0.9566326530612245
Classification Report (Best Random Forest with SMOTE):
               precision    recall  f1-score   support

           0      0.880     0.973     0.924        75
           1      0.933     0.933     0.933        60
           2      0.955     0.929     0.942       182
           3      0.973     0.973     0.973        37
           4      0.945     0.972     0.958       212
           5      0.922     0.922     0.922        77
           6      0.966     0.927     0.947       248
           7      1.000     1.000     1.000        41
           8      0.891     0.983     0.934        58
           9      0.929     0.940     0.935       167
          10      1.000     1.000     1.000        43
          11      0.880     0.786     0.830        28
          12      1.000     1.000     1.000        66
      

In [5]:
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize Borderline SMOTE

# Define the parameter grid for GridSearchCV
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_rf_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_rf = best_rf_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
classification_report_result_best_rf = classification_report(y_test, y_pred_best_rf, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Random Forest with Borderline SMOTE): {accuracy_best_rf}")
print("Classification Report (Best Random Forest with Borderline SMOTE):\n", classification_report_result_best_rf)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'max_depth': None, 'n_estimators': 200}
Accuracy (Best Random Forest with Borderline SMOTE): 0.9504373177842566
Classification Report (Best Random Forest with Borderline SMOTE):
               precision    recall  f1-score   support

           0      0.877     0.947     0.910        75
           1      0.933     0.933     0.933        60
           2      0.960     0.912     0.935       182
           3      1.000     0.973     0.986        37
           4      0.923     0.967     0.945       212
           5      0.883     0.883     0.883        77
           6      0.951     0.931     0.941       248
           7      1.000     0.951     0.975        41
           8      0.889     0.966     0.926        58
           9      0.945     0.928     0.937       167
          10      1.000     1.000     1.000        43
          11      0.909     0.714     0.800        28
          12      1.000     0.985     

Gaussian NB

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
# Define the parameter grid
param_grid = {
    'var_smoothing': [1e-9, 1.778e-7, 3.162e-5, 5.6e-3, 1]
}

# Initialize the GaussianNB classifier
gnb_classifier = GaussianNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=gnb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_gnb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_gnb = best_gnb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_gnb = accuracy_score(y_test, y_pred_best_gnb)
classification_report_result_best_gnb = classification_report(y_test, y_pred_best_gnb, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best GaussianNB with SMOTE): {accuracy_best_gnb}")
print("Classification Report (Best GaussianNB with SMOTE):\n", classification_report_result_best_gnb)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'var_smoothing': 1e-09}
Accuracy (Best GaussianNB with SMOTE): 0.22631195335276968
Classification Report (Best GaussianNB with SMOTE):
               precision    recall  f1-score   support

           0      0.800     0.053     0.100        75
           1      0.500     0.033     0.062        60
           2      0.551     0.236     0.331       182
           3      0.342     0.703     0.460        37
           4      0.358     0.325     0.341       212
           5      0.078     0.662     0.140        77
           6      0.278     0.020     0.038       248
           7      0.173     1.000     0.295        41
           8      0.422     0.655     0.514        58
           9      0.085     0.084     0.084       167
          10      1.000     0.860     0.925        43
          11      0.135     0.250     0.175        28
          12      0.269     0.682     0.386        66
          13      0.585     0

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
# Define the parameter grid
param_grid = {
    'var_smoothing': [1e-9, 1.778e-7, 3.162e-5, 5.6e-3, 1]
}

# Initialize the GaussianNB classifier
gnb_classifier = GaussianNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=gnb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_gnb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_gnb = best_gnb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_gnb = accuracy_score(y_test, y_pred_best_gnb)
classification_report_result_best_gnb = classification_report(y_test, y_pred_best_gnb, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best GaussianNB with SMOTE): {accuracy_best_gnb}")
print("Classification Report (Best GaussianNB with SMOTE):\n", classification_report_result_best_gnb)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'var_smoothing': 1e-09}
Accuracy (Best GaussianNB with SMOTE): 0.23287172011661808
Classification Report (Best GaussianNB with SMOTE):
               precision    recall  f1-score   support

           0      0.500     0.027     0.051        75
           1      0.231     0.050     0.082        60
           2      0.500     0.269     0.350       182
           3      0.308     0.757     0.438        37
           4      0.362     0.297     0.326       212
           5      0.080     0.623     0.142        77
           6      0.231     0.036     0.063       248
           7      0.191     0.927     0.317        41
           8      0.414     0.621     0.497        58
           9      0.087     0.084     0.086       167
          10      1.000     0.837     0.911        43
          11      0.074     0.143     0.098        28
          12      0.392     0.576     0.466        66
          13      0.568     0

DT

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None]
}

# Initialize the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=dt_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_dt_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_dt = best_dt_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_dt = accuracy_score(y_test, y_pred_best_dt)
classification_report_result_best_dt = classification_report(y_test, y_pred_best_dt, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Decision Tree with SMOTE): {accuracy_best_dt}")
print("Classification Report (Best Decision Tree with SMOTE):\n", classification_report_result_best_dt)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'max_depth': None}
Accuracy (Best Decision Tree with SMOTE): 0.9092565597667639
Classification Report (Best Decision Tree with SMOTE):
               precision    recall  f1-score   support

           0      0.847     0.813     0.830        75
           1      0.746     0.883     0.809        60
           2      0.918     0.863     0.890       182
           3      0.970     0.865     0.914        37
           4      0.927     0.962     0.944       212
           5      0.707     0.844     0.769        77
           6      0.949     0.907     0.928       248
           7      0.902     0.902     0.902        41
           8      0.852     0.897     0.874        58
           9      0.946     0.940     0.943       167
          10      0.976     0.930     0.952        43
          11      0.548     0.607     0.576        28
          12      0.915     0.985     0.949        66
          13      0.981     0

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None]
}

# Initialize the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=dt_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_dt_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_dt = best_dt_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_dt = accuracy_score(y_test, y_pred_best_dt)
classification_report_result_best_dt = classification_report(y_test, y_pred_best_dt, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Decision Tree with SMOTE): {accuracy_best_dt}")
print("Classification Report (Best Decision Tree with SMOTE):\n", classification_report_result_best_dt)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'max_depth': None}
Accuracy (Best Decision Tree with SMOTE): 0.9158163265306123
Classification Report (Best Decision Tree with SMOTE):
               precision    recall  f1-score   support

           0      0.775     0.827     0.800        75
           1      0.724     0.917     0.809        60
           2      0.901     0.901     0.901       182
           3      0.872     0.919     0.895        37
           4      0.947     0.929     0.938       212
           5      0.829     0.883     0.855        77
           6      0.944     0.891     0.917       248
           7      0.950     0.927     0.938        41
           8      0.823     0.879     0.850        58
           9      0.890     0.916     0.903       167
          10      1.000     1.000     1.000        43
          11      0.615     0.571     0.593        28
          12      0.970     0.970     0.970        66
          13      0.991     0

KNN


In [10]:
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assume X_final and y_encoded are your features and target after preprocessing


# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE to the training data only
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5]
}

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=knn_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_knn_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_knn = best_knn_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_knn = accuracy_score(y_test, y_pred_best_knn)
classification_report_result_best_knn = classification_report(y_test, y_pred_best_knn, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best KNN with SMOTE): {accuracy_best_knn}")
print("Classification Report (Best KNN with SMOTE):\n", classification_report_result_best_knn)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'n_neighbors': 1}
Accuracy (Best KNN with SMOTE): 0.6388483965014577
Classification Report (Best KNN with SMOTE):
               precision    recall  f1-score   support

           0      0.533     0.640     0.582        75
           1      0.508     0.533     0.520        60
           2      0.610     0.610     0.610       182
           3      0.667     0.703     0.684        37
           4      0.658     0.689     0.673       212
           5      0.321     0.455     0.376        77
           6      0.627     0.609     0.618       248
           7      0.407     0.585     0.480        41
           8      0.646     0.724     0.683        58
           9      0.602     0.581     0.591       167
          10      0.542     0.605     0.571        43
          11      0.361     0.464     0.406        28
          12      0.794     0.758     0.775        66
          13      0.742     0.659     0.698       

In [11]:
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assume X_final and y_encoded are your features and target after preprocessing


# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply Borderline SMOTE to the training data only
smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5]
}

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=knn_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_knn_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_knn = best_knn_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_knn = accuracy_score(y_test, y_pred_best_knn)
classification_report_result_best_knn = classification_report(y_test, y_pred_best_knn, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best KNN with Borderline SMOTE): {accuracy_best_knn}")
print("Classification Report (Best KNN with Borderline SMOTE):\n", classification_report_result_best_knn)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'n_neighbors': 1}
Accuracy (Best KNN with Borderline SMOTE): 0.6428571428571429
Classification Report (Best KNN with Borderline SMOTE):
               precision    recall  f1-score   support

           0      0.548     0.613     0.579        75
           1      0.508     0.550     0.528        60
           2      0.629     0.604     0.616       182
           3      0.675     0.730     0.701        37
           4      0.654     0.712     0.682       212
           5      0.343     0.455     0.391        77
           6      0.619     0.629     0.624       248
           7      0.444     0.585     0.505        41
           8      0.683     0.707     0.695        58
           9      0.608     0.605     0.607       167
          10      0.529     0.628     0.574        43
          11      0.387     0.429     0.407        28
          12      0.781     0.758     0.769        66
          13      0.723     

SVM

In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE



# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
}

# Initialize the SVM classifier
svm_classifier = SVC()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=svm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_svm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_svm = best_svm_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_svm = accuracy_score(y_test, y_pred_best_svm)
classification_report_result_best_svm = classification_report(y_test, y_pred_best_svm, digits=3)

# Print results
print(f"Best Parameters (SVM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (SVM with SMOTE): {accuracy_best_svm}")
print("Classification Report (SVM with SMOTE):\n", classification_report_result_best_svm)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters (SVM with SMOTE): {'C': 100, 'gamma': 0.1}
Accuracy (SVM with SMOTE): 0.7959183673469388
Classification Report (SVM with SMOTE):
               precision    recall  f1-score   support

           0      0.604     0.773     0.678        75
           1      0.742     0.817     0.778        60
           2      0.778     0.808     0.792       182
           3      0.842     0.865     0.853        37
           4      0.770     0.887     0.825       212
           5      0.513     0.519     0.516        77
           6      0.715     0.730     0.723       248
           7      0.844     0.927     0.884        41
           8      0.741     0.741     0.741        58
           9      0.821     0.713     0.763       167
          10      0.821     0.744     0.780        43
          11      0.522     0.429     0.471        28
          12      0.905     0.864     0.884        66
          13      0.891     0.883   

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply BorderlineSMOTE
borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = borderline_smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
}

# Initialize the SVM classifier
svm_classifier = SVC()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=svm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_svm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_svm = best_svm_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_svm = accuracy_score(y_test, y_pred_best_svm)
classification_report_result_best_svm = classification_report(y_test, y_pred_best_svm, digits=3)

# Print results
print(f"Best Parameters (SVM with BorderlineSMOTE): {grid_search.best_params_}")
print(f"Accuracy (SVM with BorderlineSMOTE): {accuracy_best_svm}")
print("Classification Report (SVM with BorderlineSMOTE):\n", classification_report_result_best_svm)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters (SVM with BorderlineSMOTE): {'C': 100, 'gamma': 0.1}
Accuracy (SVM with BorderlineSMOTE): 0.8032069970845481
Classification Report (SVM with BorderlineSMOTE):
               precision    recall  f1-score   support

           0      0.648     0.760     0.699        75
           1      0.729     0.850     0.785        60
           2      0.772     0.819     0.795       182
           3      0.842     0.865     0.853        37
           4      0.757     0.882     0.815       212
           5      0.554     0.532     0.543        77
           6      0.750     0.762     0.756       248
           7      0.860     0.902     0.881        41
           8      0.811     0.741     0.775        58
           9      0.827     0.772     0.799       167
          10      0.795     0.721     0.756        43
          11      0.520     0.464     0.491        28
          12      0.923     0.909     0.916        66
      

LR

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],  # Choose either 'lbfgs' or 'liblinear' solver
}

# Initialize the Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if necessary

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lr_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_lr_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lr = best_lr_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_lr = accuracy_score(y_test, y_pred_best_lr)
classification_report_result_best_lr = classification_report(y_test, y_pred_best_lr, digits=3)

# Print results
print(f"Best Parameters (Logistic Regression with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (Logistic Regression with SMOTE): {accuracy_best_lr}")
print("Classification Report (Logistic Regression with SMOTE):\n", classification_report_result_best_lr)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters (Logistic Regression with SMOTE): {'C': 10, 'solver': 'lbfgs'}
Accuracy (Logistic Regression with SMOTE): 0.2631195335276968
Classification Report (Logistic Regression with SMOTE):
               precision    recall  f1-score   support

           0      0.056     0.013     0.022        75
           1      0.069     0.067     0.068        60
           2      0.507     0.203     0.290       182
           3      0.290     0.784     0.423        37
           4      0.483     0.264     0.341       212
           5      0.125     0.429     0.194        77
           6      0.412     0.218     0.285       248
           7      0.158     0.927     0.270        41
           8      0.299     0.655     0.411        58
           9      0.175     0.192     0.183       167
          10      0.192     0.744     0.305        43
          11      0.133     0.464     0.206        28
          12      0.345     0.758     0

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply BorderlineSMOTE
smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],  # Choose either 'lbfgs' or 'liblinear' solver
}

# Initialize the Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if necessary

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lr_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_lr_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lr = best_lr_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_lr = accuracy_score(y_test, y_pred_best_lr)
classification_report_result_best_lr = classification_report(y_test, y_pred_best_lr, digits=3)

# Print results
print(f"Best Parameters (Logistic Regression with BorderlineSMOTE): {grid_search.best_params_}")
print(f"Accuracy (Logistic Regression with BorderlineSMOTE): {accuracy_best_lr}")
print("Classification Report (Logistic Regression with BorderlineSMOTE):\n", classification_report_result_best_lr)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters (Logistic Regression with BorderlineSMOTE): {'C': 10, 'solver': 'lbfgs'}
Accuracy (Logistic Regression with BorderlineSMOTE): 0.2478134110787172
Classification Report (Logistic Regression with BorderlineSMOTE):
               precision    recall  f1-score   support

           0      0.167     0.067     0.095        75
           1      0.065     0.067     0.066        60
           2      0.407     0.181     0.251       182
           3      0.267     0.730     0.391        37
           4      0.434     0.250     0.317       212
           5      0.168     0.558     0.258        77
           6      0.337     0.133     0.191       248
           7      0.170     0.927     0.288        41
           8      0.322     0.655     0.432        58
           9      0.157     0.186     0.170       167
          10      0.183     0.744     0.294        43
          11      0.136     0.429     0.207        28
         

XGBoost

In [16]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, None],
}

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_xgb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_xgb = best_xgb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)
classification_report_result_best_xgb = classification_report(y_test, y_pred_best_xgb, digits=3)

# Print results
print(f"Best Parameters (XGBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (XGBoost with SMOTE): {accuracy_best_xgb}")
print("Classification Report (XGBoost with SMOTE):\n", classification_report_result_best_xgb)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Parameters (XGBoost with SMOTE): {'gamma': None, 'max_depth': None}
Accuracy (XGBoost with SMOTE): 0.9577259475218659
Classification Report (XGBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.932     0.920     0.926        75
           1      0.932     0.917     0.924        60
           2      0.961     0.940     0.950       182
           3      1.000     0.946     0.972        37
           4      0.935     0.958     0.946       212
           5      0.892     0.961     0.925        77
           6      0.980     0.968     0.974       248
           7      0.976     1.000     0.988        41
           8      0.914     0.914     0.914        58
           9      0.930     0.958     0.944       167
          10      1.000     1.000     1.000        43
          11      0.750     0.750     0.750        28
          12      1.000     1.000     1.000        66
          13 

In [17]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, None],
}

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_xgb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_xgb = best_xgb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)
classification_report_result_best_xgb = classification_report(y_test, y_pred_best_xgb, digits=3)

# Print results
print(f"Best Parameters (XGBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (XGBoost with SMOTE): {accuracy_best_xgb}")
print("Classification Report (XGBoost with SMOTE):\n", classification_report_result_best_xgb)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Parameters (XGBoost with SMOTE): {'gamma': None, 'max_depth': None}
Accuracy (XGBoost with SMOTE): 0.9661078717201166
Classification Report (XGBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.909     0.933     0.921        75
           1      0.966     0.950     0.958        60
           2      0.956     0.956     0.956       182
           3      1.000     1.000     1.000        37
           4      0.944     0.958     0.951       212
           5      0.949     0.974     0.962        77
           6      0.980     0.980     0.980       248
           7      1.000     1.000     1.000        41
           8      0.931     0.931     0.931        58
           9      0.970     0.964     0.967       167
          10      1.000     1.000     1.000        43
          11      0.955     0.750     0.840        28
          12      1.000     1.000     1.000        66
          13 

In [18]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the XGBoost classifier with default parameters
xgb_classifier_baseline = XGBClassifier(random_state=42)

# Fit the classifier on the training data (assuming X_train and y_train are defined)
xgb_classifier_baseline.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb_baseline = xgb_classifier_baseline.predict(X_test)

# Evaluate the baseline model
accuracy_xgb_baseline = accuracy_score(y_test, y_pred_xgb_baseline)
classification_report_result_xgb_baseline = classification_report(y_test, y_pred_xgb_baseline, digits=3)

# Print results
print("Baseline XGBoost Model:")
print(f"Accuracy: {accuracy_xgb_baseline}")
print("Classification Report:\n", classification_report_result_xgb_baseline)


Baseline XGBoost Model:
Accuracy: 0.989067055393586
Classification Report:
               precision    recall  f1-score   support

           0      0.987     0.987     0.987        75
           1      0.984     1.000     0.992        60
           2      0.984     0.989     0.986       182
           3      1.000     0.973     0.986        37
           4      0.981     0.981     0.981       212
           5      0.975     1.000     0.987        77
           6      0.984     1.000     0.992       248
           7      1.000     1.000     1.000        41
           8      0.983     0.983     0.983        58
           9      1.000     0.982     0.991       167
          10      1.000     1.000     1.000        43
          11      1.000     0.786     0.880        28
          12      1.000     1.000     1.000        66
          13      1.000     0.996     0.998       223
          14      0.964     0.955     0.960       112
          15      1.000     0.995     0.997       194
     

AdaBoost

In [19]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity


# Define the parameter grid for AdaBoost
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.1, 1],
}

# Initialize the AdaBoost classifier
ada_classifier = AdaBoostClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=ada_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV on resampled data
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_ada_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_ada = best_ada_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_ada = accuracy_score(y_test, y_pred_best_ada)
classification_report_result_best_ada = classification_report(y_test, y_pred_best_ada, digits=3)

# Print results
print(f"Best Parameters (AdaBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (AdaBoost with SMOTE): {accuracy_best_ada}")
print("Classification Report (AdaBoost with SMOTE):\n", classification_report_result_best_ada)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters (AdaBoost with SMOTE): {'learning_rate': 0.1, 'n_estimators': 100}
Accuracy (AdaBoost with SMOTE): 0.28316326530612246
Classification Report (AdaBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.383     0.240     0.295        75
           1      0.000     0.000     0.000        60
           2      0.471     0.484     0.477       182
           3      0.604     0.784     0.682        37
           4      0.685     0.288     0.405       212
           5      0.067     0.130     0.088        77
           6      0.197     0.060     0.093       248
           7      0.528     0.683     0.596        41
           8      0.500     0.052     0.094        58
           9      0.125     0.084     0.100       167
          10      0.727     0.372     0.492        43
          11      0.120     0.714     0.206        28
          12      0.319     0.682     0.435        66
 

In [20]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity


# Define the parameter grid for AdaBoost
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.1, 1],
}

# Initialize the AdaBoost classifier
ada_classifier = AdaBoostClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=ada_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV on resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_ada_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_ada = best_ada_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_ada = accuracy_score(y_test, y_pred_best_ada)
classification_report_result_best_ada = classification_report(y_test, y_pred_best_ada, digits=3)

# Print results
print(f"Best Parameters (AdaBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (AdaBoost with SMOTE): {accuracy_best_ada}")
print("Classification Report (AdaBoost with SMOTE):\n", classification_report_result_best_ada)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters (AdaBoost with SMOTE): {'learning_rate': 0.1, 'n_estimators': 100}
Accuracy (AdaBoost with SMOTE): 0.3086734693877551
Classification Report (AdaBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.409     0.240     0.303        75
           1      0.000     0.000     0.000        60
           2      0.567     0.533     0.550       182
           3      0.724     0.568     0.636        37
           4      0.681     0.231     0.345       212
           5      0.078     0.234     0.117        77
           6      0.382     0.137     0.202       248
           7      0.788     0.634     0.703        41
           8      1.000     0.155     0.269        58
           9      0.183     0.126     0.149       167
          10      0.810     0.395     0.531        43
          11      0.175     0.714     0.282        28
          12      0.321     0.409     0.360        66
  

In [21]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the AdaBoost classifier with default parameters
ada_classifier_baseline = AdaBoostClassifier(random_state=42)

# Fit the classifier on the training data (assuming X_train and y_train are defined)
ada_classifier_baseline.fit(X_train, y_train)

# Make predictions on the test set
y_pred_ada_baseline = ada_classifier_baseline.predict(X_test)

# Evaluate the baseline model
accuracy_ada_baseline = accuracy_score(y_test, y_pred_ada_baseline)
classification_report_result_ada_baseline = classification_report(y_test, y_pred_ada_baseline, digits=3)

# Print results
print("Baseline AdaBoost Model:")
print(f"Accuracy: {accuracy_ada_baseline}")
print("Classification Report:\n", classification_report_result_ada_baseline)


Baseline AdaBoost Model:
Accuracy: 0.19533527696793002
Classification Report:
               precision    recall  f1-score   support

           0      0.000     0.000     0.000        75
           1      0.000     0.000     0.000        60
           2      0.122     0.478     0.194       182
           3      0.000     0.000     0.000        37
           4      0.198     0.844     0.321       212
           5      0.000     0.000     0.000        77
           6      0.000     0.000     0.000       248
           7      0.000     0.000     0.000        41
           8      0.000     0.000     0.000        58
           9      0.000     0.000     0.000       167
          10      0.000     0.000     0.000        43
          11      0.000     0.000     0.000        28
          12      0.000     0.000     0.000        66
          13      0.000     0.000     0.000       223
          14      0.000     0.000     0.000       112
          15      0.473     0.361     0.409       194
  

CatBoost

In [22]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Assuming X_train_resampled_smote, y_train_resampled_smote, X_test, y_test are defined

# Initialize the CatBoost classifier with default parameters
catboost_classifier = CatBoostClassifier(random_state=42)


# Fit the CatBoost classifier
catboost_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_catboost = catboost_classifier.predict(X_test)

# Evaluate the model
accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
classification_report_catboost = classification_report(y_test, y_pred_catboost, digits=3)

# Print results
print(f"Accuracy (CatBoost Baseline with SMOTE): {accuracy_catboost}")
print("Classification Report (CatBoost Baseline with SMOTE):\n", classification_report_catboost)


Learning rate set to 0.08941
0:	learn: 2.8229482	total: 397ms	remaining: 6m 36s
1:	learn: 2.6119468	total: 505ms	remaining: 4m 12s
2:	learn: 2.4468491	total: 563ms	remaining: 3m 7s
3:	learn: 2.3238602	total: 629ms	remaining: 2m 36s
4:	learn: 2.2090412	total: 704ms	remaining: 2m 20s
5:	learn: 2.1230324	total: 766ms	remaining: 2m 6s
6:	learn: 2.0393641	total: 822ms	remaining: 1m 56s
7:	learn: 1.9688408	total: 882ms	remaining: 1m 49s
8:	learn: 1.9052184	total: 952ms	remaining: 1m 44s
9:	learn: 1.8473436	total: 1.01s	remaining: 1m 40s
10:	learn: 1.7773804	total: 1.06s	remaining: 1m 35s
11:	learn: 1.7188466	total: 1.13s	remaining: 1m 33s
12:	learn: 1.6663020	total: 1.19s	remaining: 1m 30s
13:	learn: 1.6273779	total: 1.25s	remaining: 1m 27s
14:	learn: 1.5886580	total: 1.31s	remaining: 1m 26s
15:	learn: 1.5468560	total: 1.38s	remaining: 1m 24s
16:	learn: 1.5161634	total: 1.46s	remaining: 1m 24s
17:	learn: 1.4646648	total: 1.51s	remaining: 1m 22s
18:	learn: 1.4314792	total: 1.57s	remaining: 1m

LGBM

In [23]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for LGBM
param_grid = {
    'max_depth': [-1, 2, 3, 4, 5],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the LGBM classifier
lgbm_classifier = LGBMClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lgbm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Assuming X_train_resampled_smote, y_train_resampled_smote are defined
# Fit GridSearchCV on SMOTE resampled data
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_lgbm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lgbm = best_lgbm_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_lgbm = accuracy_score(y_test, y_pred_best_lgbm)
classification_report_result_best_lgbm = classification_report(y_test, y_pred_best_lgbm, digits=3)

# Print results
print(f"Best Parameters (LGBM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (LGBM with SMOTE): {accuracy_best_lgbm}")
print("Classification Report (LGBM with SMOTE):\n", classification_report_result_best_lgbm)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002807 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1486
[LightGBM] [Info] Number of data points in the train set: 40722, number of used features: 15
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -

In [24]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for LGBM
param_grid = {
    'max_depth': [-1, 2, 3, 4, 5],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the LGBM classifier
lgbm_classifier = LGBMClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lgbm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Assuming X_train_resampled_smote, y_train_resampled_smote are defined
# Fit GridSearchCV on SMOTE resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_lgbm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lgbm = best_lgbm_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_lgbm = accuracy_score(y_test, y_pred_best_lgbm)
classification_report_result_best_lgbm = classification_report(y_test, y_pred_best_lgbm, digits=3)

# Print results
print(f"Best Parameters (LGBM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (LGBM with SMOTE): {accuracy_best_lgbm}")
print("Classification Report (LGBM with SMOTE):\n", classification_report_result_best_lgbm)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006120 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1485
[LightGBM] [Info] Number of data points in the train set: 40722, number of used features: 15
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -3.091042
[LightGBM] [Info] Start training from score -